<h4>Installing missing library: lxml</h4>

In [24]:
!conda install -c conda-forge lxml --yes

Solving environment: done

# All requested packages already installed.



<h2>Preparing data</h2>

<h3>Reading Toronto Neighbourhood data from Wikiperdia</h3>

In [25]:
import pandas as pd
import lxml

In [30]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = pd.read_html(url, header=0)
df = data[0]

ImportError: lxml not found, please install it

<h3>There is not only Goelocator failing here, this whole labs also fails periodically and above code stops working</h3>
<h3>What a crap  !!!</h3>
<h4>In such circumstances I'm forced to get that data elsewhere and upload them as CSV (to be clear I used exactly the same code)</h4>

In [34]:
df = pd.read_csv('wiki-data.csv')

<h4>Initial data contains lots of empty rows (no Borough and Neighborhood)</h4>
<h5>Fortunately such rows always miss both values so it is easy to clea up</h5>

In [35]:
print(df.count())
df1 = df.dropna()
print(df1.head())
print(df1.count())

Unnamed: 0      180
Postal code     180
Borough         180
Neighborhood    103
dtype: int64
   Unnamed: 0 Postal code           Borough  \
2           2         M3A        North York   
3           3         M4A        North York   
4           4         M5A  Downtown Toronto   
5           5         M6A        North York   
6           6         M7A  Downtown Toronto   

                                   Neighborhood  
2                                     Parkwoods  
3                              Victoria Village  
4                    Regent Park / Harbourfront  
5             Lawrence Manor / Lawrence Heights  
6  Queen's Park / Ontario Provincial Government  
Unnamed: 0      103
Postal code     103
Borough         103
Neighborhood    103
dtype: int64


In [36]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df1['Borough'].unique()),
        df1.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


<h4>Installing another missing library: geopy</h4>

In [43]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0



geopy-1.21.0         | 58 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [44]:
column_names = ['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']
neighborhoods = pd.DataFrame(columns=column_names)

In [45]:
df1.keys()

Index(['Unnamed: 0', 'Postal code', 'Borough', 'Neighborhood'], dtype='object')

In [46]:
neighborhoods.keys()

Index(['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

In [47]:
# address = 'Toronto, '+df1['Neighborhood']+', Ontario, '+df1['Postal code']+', Canada'
address = 'Toronto, Parkwoods, M3A'

geolocator = Nominatim(user_agent="toronto clastering")
location = geolocator.geocode(address)
print(location)
latitude = location.latitude
longitude = location.longitude
print(latitude)
print(longitude)

Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Golden Horseshoe, Ontario, M3A 1Z5, Canada
43.7612239
-79.3239857


<h4>Geolocator service is not reliable. Sometimes it works (like above) but in most case it comes back as time out, unavailable or just return nothing</h4>

In [48]:
geolocator = Nominatim(user_agent="toronto clastering")

for data in df1:
    neighborhoods['Postal Code'] = df1['Postal code']
    neighborhoods['Borough'] = df1['Borough'] 
    neighborhoods['Neighborhood'] = df1['Neighborhood']
    address = 'Toronto, '+df1['Neighborhood']+', '+df1['Postal code']
    location = geolocator.geocode(address)
    if location is not None:
        neighborhoods['Latitude'] = location.latitude
        neighborhoods['Longitude'] = location.longitude
    
print('Extraction done')

Extraction done


<h4>In the circumstance that above code fails to work (what heppened to many times) I decided to update locations from CSV file provided by Courseera</h4>

In [40]:
geo = pd.read_csv('Geospatial_Coordinates.csv')

In [41]:
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [49]:
neighborhoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,NaN,NaN
3,M4A,North York,Victoria Village,NaN,NaN
4,M5A,Downtown Toronto,Regent Park / Harbourfront,NaN,NaN
5,M6A,North York,Lawrence Manor / Lawrence Heights,NaN,NaN
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,NaN,NaN


<h4>Need to sort neighborhoods by post code to much geolocation file (for faster and easier iteration and update)</h4>

In [50]:
neighborhoods.sort_values('Postal Code', axis = 0, ascending = True, inplace = True)
neighborhoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
9,M1B,Scarborough,Malvern / Rouge,NaN,NaN
18,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,NaN,NaN
27,M1E,Scarborough,Guildwood / Morningside / West Hill,NaN,NaN
36,M1G,Scarborough,Woburn,NaN,NaN
45,M1H,Scarborough,Cedarbrae,NaN,NaN


<h4>Now I can update Neighborhoods data frame with Latitude and Longitude values</h4>

In [51]:
i=0
success=0
for index, row in neighborhoods.iterrows():
    if row['Postal Code'] == geo.loc[i,'Postal Code']:
        row['Latitude'] = geo.loc[i,'Latitude']
        row['Longitude'] = geo.loc[i,'Longitude']
        success=success+1

    i=i+1

print('Extraxtion done'+' records updated: '+str(success))

Extraxtion done records updated: 103


In [52]:
neighborhoods

,Postal Code,Borough,Neighborhood,Latitude,Longitude
9,M1B,Scarborough,Malvern / Rouge,43.8067,-79.1944
18,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.7845,-79.1605
27,M1E,Scarborough,Guildwood / Morningside / West Hill,43.7636,-79.1887
36,M1G,Scarborough,Woburn,43.771,-79.2169
45,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
...,...,...,...,...,...
98,M9N,York,Weston,43.7069,-79.5182
107,M9P,Etobicoke,Westmount,43.6963,-79.5322
116,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...,43.6889,-79.5547
143,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...,43.7394,-79.5884


<h2> Data Analysis phase</h2>

<h4> Data are ready to process, creating a map of neighborhoods</h4>

In [53]:
import folium

In [54]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='green', fill=True, fill_color='yellow', fill_opacity=0.7, parse_html=False).add_to(toronto_map)  
    
toronto_map

<h4>Now, let's get the top 10 venues in each location within a radius of 500 meters. Due to ongoing Corona Virus crisis I don't expect many if any</h4>

First I need to assign Foursquare credencials and parameters

In [55]:
CLIENT_ID = 'A5TICW55NJDUX32LXJBD3V0E0ETCRUNMWZAUGIVBM2F1MA1F' # your Foursquare ID
CLIENT_SECRET = 'ZM21L5MF05JXJQCCTP4CLAAKBWO0CK23J430UN1YZWM1W0B1' # your Foursquare Secret
VERSION = '20180604'
radius = 500
limit = 10
# print('Your credentails:')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)

First let's try the very first location 

In [56]:
neighborhood = neighborhoods[0:1]
print(neighborhood)
latitude = neighborhood.iloc[0].at['Latitude']
longitude = neighborhoods.iloc[0].at['Longitude']

print(latitude)
print(longitude)

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, latitude, longitude, radius, limit)

url

  Postal Code      Borough     Neighborhood Latitude Longitude
9         M1B  Scarborough  Malvern / Rouge  43.8067  -79.1944
43.806686299999996
-79.19435340000001


'https://api.foursquare.com/v2/venues/explore?&client_id=A5TICW55NJDUX32LXJBD3V0E0ETCRUNMWZAUGIVBM2F1MA1F&client_secret=ZM21L5MF05JXJQCCTP4CLAAKBWO0CK23J430UN1YZWM1W0B1&v=20180604&ll=43.806686299999996,-79.19435340000001&radius=500&limit=10'

In [59]:
import requests
import json
from pandas.io.json import json_normalize

In [62]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

In [67]:
results = requests.get(url).json()
results

venues = results['response']['groups'][0]['items']    
nearby_venues = json_normalize(venues)
nearby_venues

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups
0,e-0-4bb6b9446edc76b0d771311c-0,0,"[{'summary': 'This spot is popular', 'type': '...",4bb6b9446edc76b0d771311c,Wendy’s,Morningside & Sheppard,43.807448,-79.199056,"[{'label': 'display', 'lat': 43.80744841934756...",387,CA,Toronto,ON,Canada,"[Toronto ON, Canada]","[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",0,[]


In [68]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Wendy’s,None,43.807448,-79.199056


In [69]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


<h5>that's what I thought, there is just 1 venue out there</h5>
<h5>let's iterate through all the locations for the same then (using function form lab training) </h5>

In [74]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [75]:
toronot_venues = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'])

Malvern / Rouge
Rouge Hill / Port Union / Highland Creek
Guildwood / Morningside / West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park / Ionview / East Birchmount Park
Golden Mile / Clairlea / Oakridge
Cliffside / Cliffcrest / Scarborough Village West
Birch Cliff / Cliffside West
Dorset Park / Wexford Heights / Scarborough Town Centre
Wexford / Maryvale
Agincourt
Clarks Corners / Tam O'Shanter / Sullivan
Milliken / Agincourt North / Steeles East / L'Amoreaux East
Steeles West / L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview / Henry Farm / Oriole
Bayview Village
York Mills / Silver Hills
Willowdale / Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor / Wilson Heights / Downsview North
Northwood Park / York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill / Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West / Riverdale
India Bazaar / The Beaches 

In [76]:
print(toronot_venues.shape)
toronot_venues.head()

(693, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Malvern / Rouge,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
2,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,Guildwood / Morningside / West Hill,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,Guildwood / Morningside / West Hill,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


<h5>Let's see how many Toronto venues per Neighborhood were returned</h5>

In [77]:
toronot_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
Alderwood / Long Branch,9,9,9,9,9,9
Bathurst Manor / Wilson Heights / Downsview North,10,10,10,10,10,10
Bayview Village,4,4,4,4,4,4
Bedford Park / Lawrence Manor East,10,10,10,10,10,10
...,...,...,...,...,...,...
Wexford / Maryvale,6,6,6,6,6,6
Willowdale,15,15,15,15,15,15
Woburn,3,3,3,3,3,3


<h5>Unique venue categories returned</h5>

In [82]:
print('There are {} uniques categories.'.format(len(toronot_venues['Venue Category'].unique())))

There are 175 uniques categories.


In [85]:
toronto_onehot = pd.get_dummies(toronot_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronot_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,...,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h4>What is that ? Nothing happens in Toronto right now at all? Although possible it's hard to beleive</h4>
<h5> IT has no sense so all I can do is following the way in was done for New York</h5>
<h5>Grouping venues goes first</h5>

In [86]:
toronto_onehot.shape
grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,...,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alderwood / Long Branch,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bathurst Manor / Wilson Heights / Downsview North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Bedford Park / Lawrence Manor East,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,Wexford / Maryvale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90,Willowdale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0


In [87]:
grouped.shape

(94, 175)

In [88]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot   0.2
1                     Lounge   0.2
2  Latin American Restaurant   0.2
3             Clothing Store   0.2
4               Skating Rink   0.2


----Alderwood / Long Branch----
         venue  freq
0  Pizza Place  0.22
1     Pharmacy  0.11
2          Gym  0.11
3         Pool  0.11
4  Coffee Shop  0.11


----Bathurst Manor / Wilson Heights / Downsview North----
                       venue  freq
0                Coffee Shop   0.2
1                       Bank   0.1
2  Middle Eastern Restaurant   0.1
3              Deli / Bodega   0.1
4             Ice Cream Shop   0.1


----Bayview Village----
                 venue  freq
0                 Bank  0.25
1  Japanese Restaurant  0.25
2   Chinese Restaurant  0.25
3                 Café  0.25
4          Yoga Studio  0.00


----Bedford Park / Lawrence Manor East----
                     venue  freq
0              Coffee Shop   0.2
1               Restaurant   0.

In [91]:
import numpy as np
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [93]:
num_top_venues = 3 # it was initially 10 but returned numbers were mostly 1 or 2
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Agincourt,Latin American Restaurant,Clothing Store,Lounge
1,Alderwood / Long Branch,Pizza Place,Coffee Shop,Pharmacy
2,Bathurst Manor / Wilson Heights / Downsview North,Coffee Shop,Ice Cream Shop,Bridal Shop
3,Bayview Village,Café,Japanese Restaurant,Bank
4,Bedford Park / Lawrence Manor East,Coffee Shop,Comfort Food Restaurant,Italian Restaurant


<h2>Clastering Neighborhoods</h2>
<h3>The map appears to be pretty regular, let's start form 8 clasters</h3>

In [96]:
from sklearn.cluster import KMeans

In [108]:
kclusters = 3
clustering = grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering)
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [111]:
# neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
merged = neighborhoods
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
9,M1B,Scarborough,Malvern / Rouge,43.8067,-79.1944,3.0,Fast Food Restaurant,Women's Store,Event Space
18,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.7845,-79.1605,5.0,Golf Course,Bar,Event Space
27,M1E,Scarborough,Guildwood / Morningside / West Hill,43.7636,-79.1887,2.0,Electronics Store,Medical Center,Intersection
36,M1G,Scarborough,Woburn,43.771,-79.2169,1.0,Coffee Shop,Korean Restaurant,Women's Store
45,M1H,Scarborough,Cedarbrae,43.7731,-79.2395,2.0,Lounge,Bakery,Gas Station


In [103]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [118]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3> Does it actually the same map as before? It doesn't matter how many Ks I use the maps is always the same <h3>

That's it I suppose but I feel puny about that